In [1]:
from openai import OpenAI  
import pandas as pd

In [ ]:
client = OpenAI(
    api_key="API_KEY", 
    base_url="https://api.upstage.ai/v1/solar"
)

def augment_text(text):
    try:
        response = client.chat.completions.create(
            model="solar-1-mini-chat",
            messages=[
                {
                    "role": "system",
                    "content": "다음 대화를 바탕으로 자연스러운 한국어 대화를 다양하게 변형해 주세요. summary는 대화의 가장 중요한 정보를 전달하고 대화 길이의 20% 이내로 간략하게 요약되어야하며 대화 내에서 중요한 명명된 개체를 보존하고 관찰자의 관점에서 작성되어야하고 은어나 약어 없이 공식적으로 사용되는 언어로 작성 되어야 합니다."
                },
                {
                    "role": "user",
                    "content": text
                }
            ],
            max_tokens=150,  
            n=1,  
            temperature=0.7,
        )
        augmented_text = response.choices[0].message.content.strip()
        return augmented_text
    except Exception as e:
        print(f"Error augmenting text: {e}")
        return None

df = pd.read_csv("../data/train.csv")

augmented_data = []

for index, row in df.iterrows():
    original_dialogue = row['dialogue']
    augmented_dialogue = augment_text(original_dialogue)
    
    if augmented_dialogue:
        new_row = {
            'fname': row['fname'],
            'dialogue': augmented_dialogue,
            'summary': row['summary'], 
            'topic': row['topic']
        }
        augmented_data.append(new_row)

augmented_df = pd.DataFrame(augmented_data)

augmented_df.to_csv('augmented_train.csv', index=False)

print("데이터 증강 및 저장이 완료되었습니다.")